<a href="https://colab.research.google.com/github/Rmm6699/HW/blob/master/%E6%88%90%E5%A4%A7%E6%B0%B8%E8%B1%90AI%E4%BA%BA%E6%89%8D%E5%9F%B9%E8%82%B2%E8%A8%88%E7%95%AB_2022_10_28_prectice_112512.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio===0.12.1+cu113 -f https://download.pytorch.org/whl/cu113/torch_stable.html
#pip install Pillow   #PIL

In [1]:
import os
import numpy as np         # Package that offers comprehensive mathematical functions.
from PIL import Image       # An imaging Library , we use it to load and process the image.
from matplotlib import cm
    
import torch
import torchvision
from torchvision import transforms
import torch.nn.functional as F # transform output to probability.
import torchvision.models as models

In [3]:
print("torch.__version__ : ",torch.__version__)
print("torchvision.__version__:",torchvision.__version__)

torch.__version__ :  1.12.1+cu113
torchvision.__version__: 0.13.1+cu113


In [4]:
# connect to google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
# loading pre-train model (AlexNet) from torchvision
model = models.alexnet(pretrained = True)
print(model)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=AlexNet_Weights.IMAGENET1K_V1`. You can also use `weights=AlexNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/alexnet-owt-7be5be79.pth" to /root/.cache/torch/hub/checkpoints/alexnet-owt-7be5be79.pth


  0%|          | 0.00/233M [00:00<?, ?B/s]

AlexNet(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
    (1): ReLU(inplace=True)
    (2): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
    (4): ReLU(inplace=True)
    (5): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (7): ReLU(inplace=True)
    (8): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): ReLU(inplace=True)
    (10): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (avgpool): AdaptiveAvgPool2d(output_size=(6, 6))
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=9216, out_features=4096, bias=True)
 

In [6]:
#自選一張圖片測試
https://www.kaggle.com/datasets/arjunashok33/miniimagenet
# read image
image = Image.open('./drive/MyDrive/Practice1/n02099601_100.jpg')
display(image)

# RGB
pix = image.load()
print(pix[100,100])

FileNotFoundError: ignored

In [ ]:
#這邊提供三張sample images: https://www.dropbox.com/t/Kwnjronf3rXCNRDn

In [7]:
#defite a transform function
img_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Resize((224, 224)),  
    transforms.Normalize(
            mean=[0.485, 0.456, 0.406], 
            std=[0.229, 0.224, 0.225])])

In [8]:
image_tensor = img_transforms(image)
image_tensor = image_tensor.unsqueeze(0)
image_tensor.shape

NameError: ignored

In [ ]:
output = model(image_tensor)
print(output.shape)

torch.Size([1, 1000])


In [ ]:
output = output.squeeze()
_, indices = torch.sort(output, descending=True)
probs = F.softmax(output, dim=-1)
for index in indices[:5]:
        print(f"Label {index}:  ({probs[index].item():.2f})")

Label 207:  (0.87)
Label 213:  (0.10)
Label 852:  (0.02)
Label 244:  (0.00)
Label 219:  (0.00)


In [ ]:
class sample_dataset(torch.utils.data.Dataset):
  def __init__(self, FileList, root='./', image_size=224, mode='Train'):
    super(sample_dataset, self).__init__()
    self.fns = FileList
    #in this sample ,all of our picture is golden retriever , which is label 207.
    self.y =  [207]*len(FileList)
    self.mode = mode
    self.root = root
    self.image_size = image_size
    self.mean_transform = transforms.Compose(
    [
        transforms.Resize((image_size, image_size)),
        transforms.ToTensor(),
        transforms.Normalize(mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225))
    ])
    self.transform = transforms.Compose(
    [
        transforms.Resize((image_size, image_size)),
        transforms.ColorJitter(brightness=0.4, contrast=0.4, saturation=0.4, hue=0.4),
        transforms.ToTensor(),
        transforms.Normalize(mean = (0.485, 0.456, 0.406), std = (0.229, 0.224, 0.225))
    ])
    self.n_images = len(self.fns)
        
  def __getitem__(self, index):
    im = Image.open(self.root + self.fns[index])
    if self.mode=='Train':
      im = self.transform(im)
    else:
      im = self.mean_transform(im)
    return im,self.y[index]

  def __len__(self):
    return self.n_images

In [ ]:
#the path to image
train_root = './drive/MyDrive/Practice1/n02099601/'
#search all image name.
train_filename = os.listdir(train_root) 
batch_size = 4
train_dataset = sample_dataset(FileList=train_filename,root=train_root,image_size=224,mode="test")
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)

In [ ]:
#4 means four images in this iteration.
dataiter = iter(train_loader)
data = dataiter.next()
data[0].shape

torch.Size([4, 3, 224, 224])

In [ ]:
output = model(data[0])
output = output.squeeze()
print(output.shape)

torch.Size([4, 1000])


In [ ]:
_, indices = torch.sort(output, descending=True)
probs = F.softmax(output, dim=-1)
for i in range(batch_size):
  print(i+1,"picture")
  for index in indices[i,:5]:
    print(f"Label {index}:  ({probs[i,index].item():.2f})")

1 picture
Label 207:  (0.90)
Label 219:  (0.05)
Label 852:  (0.01)
Label 213:  (0.01)
Label 260:  (0.00)
2 picture
Label 207:  (0.84)
Label 208:  (0.10)
Label 216:  (0.02)
Label 215:  (0.01)
Label 220:  (0.01)
3 picture
Label 169:  (0.52)
Label 207:  (0.12)
Label 248:  (0.05)
Label 244:  (0.05)
Label 250:  (0.05)
4 picture
Label 169:  (0.62)
Label 160:  (0.23)
Label 176:  (0.04)
Label 222:  (0.03)
Label 207:  (0.02)


In [ ]:
#top1 and top5 accuracy:
def evaluteTop1(model, loader):
  model.eval() 
  correct = 0
  total = len(loader.dataset)
  for x,y in loader:
    with torch.no_grad():
      logits = model(x)
      pred = logits.argmax(dim=1)
      correct += torch.eq(pred, y).sum().float().item()
  return correct / total


def evaluteTop5(model, loader):
  model.eval()
  correct = 0
  total = len(loader.dataset)
  for x, y in loader:
    with torch.no_grad():
      logits = model(x)
      maxk = max((1,5))
      y_resize = y.view(-1,1)
      _, pred = logits.topk(maxk, 1, True, True)
      correct += torch.eq(pred, y_resize).sum().float().item()
  return correct / total

In [ ]:
evaluteTop1(model,train_loader)

0.74

In [ ]:
evaluteTop5(model,train_loader)

0.98